In [4]:

import os
import common_util
from tqdm import notebook

splited_sentences = []

BASE_FOLDER = r"E:\DATA\copus\NIKL_NEWSPAPER_v2.0\data_kss"

all_lines = []
folders = os.listdir(BASE_FOLDER)
for folder in folders:
    folder_path = os.path.join(BASE_FOLDER, folder)
    files = os.listdir(folder_path)

    files = notebook.tqdm(files)
    for file in files:
        file_path = os.path.join(folder_path, file)
        lines = common_util.read_lines(file_path)
        # print(len(lines))
        all_lines.extend(lines)

print(len(all_lines))


  0%|          | 0/90000 [00:00<?, ?it/s]

2247441


In [5]:
from sklearn.model_selection import train_test_split

# results = splited_sentences
results = all_lines
test_size = int(len(results) * 0.1)
train, val = train_test_split(results, test_size=test_size, random_state=111)
train, test = train_test_split(train, test_size=test_size, random_state=111)


def write_dataset(file_name, dataset):
    with open(
        os.path.join("data", file_name), mode="w", encoding="utf-8"
    ) as f:
        for data in dataset:
            f.write(data)
            f.write("\n")

write_dataset("train.txt", train)
write_dataset("val.txt", val)
write_dataset("test.txt", test)

In [7]:
from typing import Callable, List, Tuple
from torch.utils.data import Dataset

class CorpusDataset(Dataset):
    def __init__(self, data_path: str):
        self.sentences = []
        self.slot_labels = ["UNK", "PAD", "B", "I"]

        self._load_data(data_path)

    def _load_data(self, data_path: str):
        """data를 file에서 불러온다.

        Args:
            data_path: file 경로
        """
        with open(data_path, mode="r", encoding="utf-8") as f:
            lines = f.readlines()
            self.sentences = [line.split() for line in lines]

    def _get_tags(self, sentence: List[str]) -> List[str]:
        """문장에 대해 띄어쓰기 tagging을 한다.
        character 단위로 분리하여 BI tagging을 한다.

        Args:
            sentence: 문장

        Retrns:
            문장의 각 토큰에 대해 tagging한 결과 리턴
            ["B", "I"]
        """

        tags = []
        for word in sentence:
            for i in range(len(word)):
                if i == 0:
                    tags.append("B")
                else:
                    tags.append("I")
        return tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = "".join(self.sentences[idx])
        sentence = [s for s in sentence]
        tags = self._get_tags(self.sentences[idx])
        tags = [self.slot_labels.index(t) for t in tags]

        (
            input_ids,
            attention_mask,
            token_type_ids,
            slot_label_ids, 
        ) = self.transform(sentence, tags)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

TabError: inconsistent use of tabs and spaces in indentation (1957826071.py, line 47)